<a href="https://colab.research.google.com/github/sourcecode369/deep-natural-language-processing/blob/master/huggingface/Hugging_Face_Usage_Guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

import transformers

     |████████████████████████████████| 552kB 9.0MB/s 
     |████████████████████████████████| 870kB 60.3MB/s 
     |████████████████████████████████| 1.0MB 91kB/s 
     |████████████████████████████████| 3.7MB 36.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=e410a28b9ffcec2db1418ec921d1fb87eea76ef3020e773e54bc9d4622a48c40
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
import tokenizers

In [0]:
from transformers import pipeline

### Sequence Classification

In [5]:
nlp = pipeline("sentiment-analysis")

In [6]:
nlp("I hate you")

[{'label': 'NEGATIVE', 'score': 0.9991129}]

In [7]:
nlp("I love you")

[{'label': 'POSITIVE', 'score': 0.99986565}]

In [8]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf 

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")

classes = ['not paraphrase', 'is paraphrase']

sequence_0 = "Aritificial Intelligence is the new fuel."
sequence_1 = "Tesla uses Artificial Intelligence in their Self Driving Cars"
sequence_2 = "Machine learning is empowering Artificial Intelligence"

paraphrase1 = tokenizer.encode_plus(sequence_0, sequence_2, return_tensors="tf")
paraphrase2 = tokenizer.encode_plus(sequence_0, sequence_1, return_tensors="tf")

In [9]:
paraphrase1

{'attention_mask': <tf.Tensor: shape=(1, 22), dtype=int32, numpy=
 array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       dtype=int32)>,
 'input_ids': <tf.Tensor: shape=(1, 22), dtype=int32, numpy=
 array([[  101, 10789,  3121, 21361,  2916,  7829,  1110,  1103,  1207,
          4251,   119,   102,  7792,  3776,  1110,  9712, 26703,  2051,
         19814,  2916,  7829,   102]], dtype=int32)>,
 'token_type_ids': <tf.Tensor: shape=(1, 22), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       dtype=int32)>}

In [10]:
paraphrase2

{'attention_mask': <tf.Tensor: shape=(1, 25), dtype=int32, numpy=
 array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]], dtype=int32)>,
 'input_ids': <tf.Tensor: shape=(1, 25), dtype=int32, numpy=
 array([[  101, 10789,  3121, 21361,  2916,  7829,  1110,  1103,  1207,
          4251,   119,   102, 12008, 26597,  2745,  2051, 19814,  2916,
          7829,  1107,  1147,  9434, 26010, 16644,   102]], dtype=int32)>,
 'token_type_ids': <tf.Tensor: shape=(1, 25), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]], dtype=int32)>}

In [11]:
tokenizer.decode(paraphrase1['input_ids'][0])

'[CLS] Aritificial Intelligence is the new fuel. [SEP] Machine learning is empowering Artificial Intelligence [SEP]'

In [12]:
tokenizer.decode(paraphrase2['input_ids'][0])

'[CLS] Aritificial Intelligence is the new fuel. [SEP] Tesla uses Artificial Intelligence in their Self Driving Cars [SEP]'

In [13]:
paraphrase_classification_logits1 = model(paraphrase1)[0]
paraphrase_classification_logits1

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 0.89733267, -0.41277018]], dtype=float32)>

In [14]:
paraphrase_classification_logits2 = model(paraphrase2)[0]
paraphrase_classification_logits2

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 1.2522348, -1.5805088]], dtype=float32)>

In [0]:
paraphrase_results_1 = tf.nn.softmax(paraphrase_classification_logits1, axis=1).numpy()[0] 
paraphrase_results_2 = tf.nn.softmax(paraphrase_classification_logits2, axis=1).numpy()[0] 

In [16]:
print("Should be paraphrase")
for i in range(len(classes)):
    print(f"{classes[i]}: {round(paraphrase_results_1[i] * 100)}%")

print("\nShould not be paraphrase")
for i in range(len(classes)):
    print(f"{classes[i]}: {round(paraphrase_results_2[i] * 100)}%")

Should be paraphrase
not paraphrase: 79.0%
is paraphrase: 21.0%

Should not be paraphrase
not paraphrase: 94.0%
is paraphrase: 6.0%


### Extractive Question Answering

In [17]:
from transformers import pipeline

nlp = pipeline("question-answering")

context = r"""
Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a
question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune
a model on a SQuAD task, you may leverage the `run_squad.py`.
"""

print(nlp(question="What is extractive question answering?", context=context))
print(nlp(question="What is a good example of a question answering dataset?", context=context))

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2639.59it/s]


{'score': 0.6222320884473547, 'start': 34, 'end': 96, 'answer': 'the task of extracting an answer from a text given a question.'}


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 5511.57it/s]


{'score': 0.5115286494260545, 'start': 147, 'end': 161, 'answer': 'SQuAD dataset,'}


In [18]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

text = r"""
🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
TensorFlow 2.0 and PyTorch.
"""

questions = [
    "How many pretrained models are available in Transformers?",
    "What does Transformers provide?",
    "Transformers provides interoperability between which frameworks?",
]

In [20]:
for question in questions:
    inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]
    print("Input IDs: ",input_ids)
    text_tokens = tokenizer.decode(input_ids)
    print("Text tokens: ",text_tokens)
    answer_start_scores, answer_end_scores = model(**inputs)

    answer_start = torch.argmax(
        answer_start_scores
    )  # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

Input IDs:  [101, 2129, 2116, 3653, 23654, 2098, 4275, 2024, 2800, 1999, 19081, 1029, 102, 100, 19081, 1006, 3839, 2124, 2004, 1052, 22123, 2953, 2818, 1011, 19081, 1998, 1052, 22123, 2953, 2818, 1011, 3653, 23654, 2098, 1011, 14324, 1007, 3640, 2236, 1011, 3800, 4294, 2015, 1006, 14324, 1010, 14246, 2102, 1011, 1016, 1010, 23455, 1010, 28712, 2213, 1010, 4487, 16643, 23373, 1010, 28712, 7159, 1529, 1007, 2005, 3019, 2653, 4824, 1006, 17953, 2226, 1007, 1998, 3019, 2653, 4245, 1006, 17953, 2290, 1007, 2007, 2058, 3590, 1009, 3653, 23654, 2098, 4275, 1999, 2531, 1009, 4155, 1998, 2784, 6970, 25918, 8010, 2090, 23435, 12314, 1016, 1012, 1014, 1998, 1052, 22123, 2953, 2818, 1012, 102]
Text tokens:  [CLS] how many pretrained models are available in transformers? [SEP] [UNK] transformers ( formerly known as pytorch - transformers and pytorch - pretrained - bert ) provides general - purpose architectures ( bert, gpt - 2, roberta, xlm, distilbert, xlnet … ) for natural language understanding 

In [23]:
for k, v in inputs.items():
  print("{} : {}".format(k,v))

input_ids : tensor([[  101, 19081,  3640,  6970, 25918,  8010,  2090,  2029,  7705,  2015,
          1029,   102,   100, 19081,  1006,  3839,  2124,  2004,  1052, 22123,
          2953,  2818,  1011, 19081,  1998,  1052, 22123,  2953,  2818,  1011,
          3653, 23654,  2098,  1011, 14324,  1007,  3640,  2236,  1011,  3800,
          4294,  2015,  1006, 14324,  1010, 14246,  2102,  1011,  1016,  1010,
         23455,  1010, 28712,  2213,  1010,  4487, 16643, 23373,  1010, 28712,
          7159,  1529,  1007,  2005,  3019,  2653,  4824,  1006, 17953,  2226,
          1007,  1998,  3019,  2653,  4245,  1006, 17953,  2290,  1007,  2007,
          2058,  3590,  1009,  3653, 23654,  2098,  4275,  1999,  2531,  1009,
          4155,  1998,  2784,  6970, 25918,  8010,  2090, 23435, 12314,  1016,
          1012,  1014,  1998,  1052, 22123,  2953,  2818,  1012,   102]])
token_type_ids : tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1,